# Payoff Calendar spread

Short T court, long T long. Payoff jambe longue, prime BS approx.


In [ ]:

import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
import sys
from pathlib import Path
from IPython.display import display, Markdown

_base = Path.cwd().resolve()
for extra in (_base, _base.parent, _base.parent.parent, _base.parent.parent.parent):
    candidate = extra / "scripts"
    if candidate.exists() and str(candidate) not in sys.path:
        sys.path.insert(0, str(candidate))
        break
    candidate = extra / "notebooks" / "scripts"
    if candidate.exists() and str(candidate) not in sys.path:
        sys.path.insert(0, str(candidate))
        break
plt.style.use('seaborn-v0_8-darkgrid')

from pricing import fetch_spy_history, view_calendar_spread
close_spy = fetch_spy_history()
spot_ref = float(close_spy.iloc[-1])
K = spot_ref
T_short = 0.25
T_long = 0.75


## Payoff interactif
S0, K, T court/long, sigma, r.


In [ ]:

sigma_slider = widgets.FloatSlider(value=0.2, min=0.01, max=1.0, step=0.01, description='Sigma')
r_slider = widgets.FloatSlider(value=0.02, min=-0.05, max=0.1, step=0.005, description='r')
T_slider = widgets.FloatSlider(value=1.0, min=0.05, max=2.0, step=0.05, description='T')

spot_slider = widgets.FloatSlider(value=spot_ref, min=spot_ref*0.6, max=spot_ref*1.4, step=1.0, description='S0')
strike_slider = widgets.FloatSlider(value=K, min=spot_ref*0.6, max=spot_ref*1.4, step=1.0, description='K')
t_short_slider = widgets.FloatSlider(value=T_short, min=0.05, max=1.0, step=0.05, description='T short')
t_long_slider = widgets.FloatSlider(value=T_long, min=0.1, max=2.0, step=0.05, description='T long')
output = widgets.Output()

def _update(change=None):
    with output:
        output.clear_output()
        s0 = spot_slider.value
        k = strike_slider.value
        t_short = t_short_slider.value
        t_long = max(t_long_slider.value, t_short + 0.01)

        fig_ts, ax_ts = plt.subplots(figsize=(8, 3))
        ax_ts.plot(close_spy.index, close_spy.values, label='SPY close (1y)')
        ax_ts.axhline(k, color='gray', linestyle='--', label=f'Strike = {k:.2f}')
        ax_ts.set_ylabel('Prix')
        ax_ts.set_title('Clotures SPY (strike en pointillé)')
        ax_ts.legend(loc='best')
        fig_ts.autofmt_xdate()
        plt.show()
        plt.close(fig_ts)

        view_dyn = view_calendar_spread(s0, k, t_short, t_long, r=r_slider.value, q=0.0, sigma=sigma_slider.value, option_type='call')
        premium = float(view_dyn.get('premium', 0.0))
        s_grid = view_dyn['s_grid']
        payoff_grid = view_dyn['payoff']
        pnl_grid = view_dyn['pnl']
        payoff_s0 = float(np.interp(s0, s_grid, payoff_grid))
        pnl_s0 = payoff_s0 - premium

        fig, ax = plt.subplots(figsize=(7,4))
        ax.plot(s_grid, payoff_grid, label='Payoff (long T long)')
        ax.plot(s_grid, pnl_grid, label='P&L net', color='darkorange')
        ax.axvline(k, color='gray', linestyle='--', label=f'K = {k:.2f}')
        ax.axvline(s0, color='crimson', linestyle='-.', label=f'S0 = {s0:.2f}')
        ax.axhline(0, color='black', linewidth=0.8)
        ax.legend(loc='best')
        ax.set_xlabel('Spot')
        ax.set_ylabel('Payoff / P&L')
        ax.set_title('Calendar spread')
        plt.show()
        plt.close(fig)

        msg = f"""Prime ~ {premium:.4f}

- Payoff @ S0 = {payoff_s0:.4f}
- P&L net = {pnl_s0:.4f}"""
        display(Markdown(msg))

for sl in (spot_slider, strike_slider, t_short_slider, t_long_slider, sigma_slider, r_slider, T_slider):
    sl.observe(_update, names='value')
_update()
display(widgets.VBox([spot_slider, strike_slider, t_short_slider, t_long_slider, sigma_slider, r_slider, T_slider, output]))
